In [11]:
# File PATH
file_path = "C:/Users/Jiawei Qi/MathsSelfLearnModel/dataset/SeniorMaths/train_data"

# Getting Data
sentences = []
labels = []
with open(file_path, "r", encoding="utf-8") as f:
    words = []
    tags = []
    for line in f:
        line = line.strip()
        if not line:  # end of a sentence
            if words:
                sentences.append(words)
                labels.append(tags)
                words, tags = [], []
        else:
            try:
                word, tag = line.split()
                words.append(word)
                tags.append(tag)
            except ValueError:
                continue  
# Get the last sentence(if it doesn't end with an empty line)
if words:
    sentences.append(words)
    labels.append(tags)



In [12]:
from collections import Counter

# Calculate tag distribution
tag_counts = Counter(tag for label_seq in labels for tag in label_seq)

# Print the tag distribution
print("Tags Distribution：")
for tag, count in tag_counts.items():
    print(f"{tag}: {count}")


Tags Distribution：
O: 65170
B-KNOW: 11132
I-KNOW: 20198
B-PRIN: 477
I-PRIN: 1481


In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

all_tags = [tag for seq in labels for tag in seq]
classes = list(set(all_tags))

print('all_tags', all_tags)
print('classes',classes)
# class_weights = compute_class_weight(class_weight='balanced', classes=np.array(classes), y=all_tags)
# class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
# print(class_weight_dict)

# Create word and label indices
all_words = set(w for s in sentences for w in s)
all_labels = set(t for l in labels for t in l)
print('all_words',all_words)
print('all_label',all_labels)

word2idx = {w: i + 2 for i, w in enumerate(all_words)}
word2idx["PAD"] = 0
word2idx["UNK"] = 1

label2idx = {t: i for i, t in enumerate(sorted(all_labels))}
print('word2idx',word2idx)
print('label2idx',label2idx)

all_tags ['O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [14]:
import numpy as np

# GloVe file path
glove_path = "C:/Users/Jiawei Qi/MathsSelfLearnModel/dataset/SeniorMaths/wiki.zh.glove.Mode"
embedding_dim = 300
embedding_index = {}

#Get GloVe embeddings
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.strip().split()
        if len(values) < embedding_dim + 1:
            continue
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector




In [15]:

# Construct the embedding_matrix, using the GloVe vector if available, otherwise a random vector
embedding_matrix = np.random.uniform(-0.05, 0.05, (len(word2idx), embedding_dim))
for word, idx in word2idx.items():
    if word in embedding_index:
        embedding_matrix[idx] = embedding_index[word]


embedding_matrix

array([[ 0.03635976, -0.00491514, -0.03743848, ..., -0.02004247,
        -0.02898045, -0.00516267],
       [-0.03704241,  0.00220845,  0.01411116, ...,  0.04103519,
         0.04264612,  0.03734027],
       [-0.49308801,  0.079197  ,  0.226615  , ...,  0.47473499,
        -0.44268399, -0.009387  ],
       ...,
       [ 0.32905501, -0.01876   ,  0.063118  , ...,  0.39789501,
        -0.115555  , -0.077041  ],
       [ 0.124552  ,  0.052212  , -0.44366899, ..., -0.469028  ,
        -0.434569  , -0.48319   ],
       [-0.54293299,  0.00853   , -0.075128  , ..., -0.109536  ,
        -0.16059899,  0.241577  ]])

In [16]:


# 正Calculate class weights for the labels
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.array(list(label2idx.keys())),
                                     y=all_tags)

class_weight_dict = {
    label2idx[label]: weight
    for label, weight in zip(label2idx.keys(), class_weights)
}
print("Correct class weight：", class_weight_dict)

# longest sentence
MAX_LEN = max(len(s) for s in sentences)

# word2idx and label2idx
X = [[word2idx.get(w, word2idx["UNK"]) for w in s] for s in sentences]
print(X)
y = [[label2idx[t] for t in l] for l in labels]
print(y)
# padding
X = pad_sequences(X, maxlen=MAX_LEN, padding='post', value=word2idx["PAD"])
y = pad_sequences(y, maxlen=MAX_LEN, padding='post', value=label2idx["O"])

Correct class weight： {0: 1.7689184333453107, 1: 41.28218029350105, 2: 0.9749282107139321, 3: 13.296151249155976, 4: 0.30215743440233234}
[[13, 635, 1060, 162, 500, 155, 423, 965, 642, 212, 807, 388, 225, 616, 602, 965, 588, 130, 800, 807, 859, 588, 500, 155, 423, 965, 642, 212, 807, 225, 922, 519, 211, 14, 965, 833], [500, 155, 1054, 965, 780, 791, 1069, 1054, 856, 876, 749, 643, 712, 557, 542, 588, 45, 749, 500, 155, 965, 533, 405, 1037, 965, 859, 615, 800, 807, 272, 751, 212, 500, 155, 1054, 661, 533, 86, 421, 965, 500, 155, 1054, 305, 423, 965, 162, 358, 588, 939, 500, 155, 807, 579, 213, 162, 965, 13, 923, 4, 394, 79, 588, 500, 155, 578, 965, 579, 4, 394, 79, 244, 690, 581, 642, 212, 833, 14, 981, 965, 500, 155, 13, 32, 415, 162, 358, 581, 752, 591, 13, 635, 251, 52, 635, 213, 162, 965, 642, 212, 1068, 393, 97, 965, 970, 269, 833], [500, 155, 807, 252, 98, 623, 380, 227, 87, 162, 772, 247, 965, 98, 269, 965, 251, 102, 975, 965, 437, 975, 1039, 654, 233, 97, 965, 500, 269, 833, 691

In [17]:
print(len(X))
print(X[0])

1953
[  13  635 1060  162  500  155  423  965  642  212  807  388  225  616
  602  965  588  130  800  807  859  588  500  155  423  965  642  212
  807  225  922  519  211   14  965  833    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [18]:
print(len(y))
print(y[0])

1953
[4 4 4 4 0 2 4 4 0 2 4 4 4 4 4 4 4 4 4 4 4 4 0 2 4 4 0 2 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4]


In [19]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

class BiLSTMCRFModel(tf.keras.Model):
    def __init__(self, vocab_size, tag_size, embedding_matrix=None, class_weights=None,
                 embedding_dim=300, lstm_units=128):
        super(BiLSTMCRFModel, self).__init__()
        self.num_tags = tag_size
        self.class_weights = class_weights
        self.embedding_dim = embedding_dim

        if embedding_matrix is not None:
            self.embedding = layers.Embedding(
                input_dim=vocab_size,
                output_dim=embedding_dim,
                weights=[embedding_matrix],
                trainable=True,         
                mask_zero=True
            )
        else:
            self.embedding = layers.Embedding(
                input_dim=vocab_size,
                output_dim=embedding_dim,
                mask_zero=True
            )

        self.bilstm = layers.Bidirectional(layers.LSTM(units=lstm_units, return_sequences=True))
        self.dense = layers.Dense(tag_size)
        self.transition_params = tf.Variable(tf.random.uniform(shape=(tag_size, tag_size)))
        self.idx2label = None  
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.bilstm(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        logits = self.dense(x)
        return logits

    def train_step(self, data):
        x, y_true = data
        with tf.GradientTape() as tape:
            logits = self(x, training=True)
            log_likelihood, _ = tfa.text.crf_log_likelihood(
                logits, y_true, tf.math.count_nonzero(x, axis=1),
                transition_params=self.transition_params
            )
            mask = tf.cast(tf.math.not_equal(y_true, 0), tf.float32)

            if self.class_weights:
                weights = tf.gather(tf.constant(self.class_weights, dtype=tf.float32), y_true)
                log_likelihood *= tf.reduce_sum(weights * mask, axis=1) / tf.reduce_sum(mask, axis=1)

            loss = -tf.reduce_mean(log_likelihood)

        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        viterbi_seq, _ = tfa.text.crf_decode(logits, self.transition_params,
                                             tf.math.count_nonzero(x, axis=1))
        matches = tf.boolean_mask(tf.equal(viterbi_seq, y_true), tf.cast(mask, tf.bool))
        accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
        return {"loss": loss, "accuracy": accuracy}

    def test_step(self, data):
        x, y_true = data
        logits = self(x, training=False)
        log_likelihood, _ = tfa.text.crf_log_likelihood(
            logits, y_true, tf.math.count_nonzero(x, axis=1),
            transition_params=self.transition_params
        )
        loss = -tf.reduce_mean(log_likelihood)

        viterbi_seq, _ = tfa.text.crf_decode(logits, self.transition_params,
                                             tf.math.count_nonzero(x, axis=1))
        mask = tf.cast(tf.math.not_equal(y_true, 0), tf.bool)
        matches = tf.boolean_mask(tf.equal(viterbi_seq, y_true), mask)
        accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
        return {"loss": loss, "accuracy": accuracy}


In [20]:
# Convert class weights to a list for the model
class_weights_array = [class_weight_dict[i] for i in range(len(label2idx))]

model = BiLSTMCRFModel(
    vocab_size=len(word2idx),
    tag_size=len(label2idx),
    class_weights=class_weights_array  
)
model.compile(optimizer='adam')
model.fit(X, y, batch_size=32, epochs=20, validation_split=0.1)


Epoch 1/20
55/55 [==============================] - 20s 227ms/step - loss: 24.5573 - accuracy: 0.2317 - val_loss: 31.4663 - val_accuracy: 0.4394
Epoch 2/20
55/55 [==============================] - 10s 181ms/step - loss: 11.1202 - accuracy: 0.2678 - val_loss: 17.1286 - val_accuracy: 0.4579
Epoch 3/20
55/55 [==============================] - 10s 191ms/step - loss: 6.9958 - accuracy: 0.2774 - val_loss: 12.1702 - val_accuracy: 0.4663
Epoch 4/20
55/55 [==============================] - 11s 198ms/step - loss: 5.2044 - accuracy: 0.2827 - val_loss: 11.3524 - val_accuracy: 0.4747
Epoch 5/20
55/55 [==============================] - 13s 234ms/step - loss: 4.3546 - accuracy: 0.2855 - val_loss: 10.7057 - val_accuracy: 0.4747
Epoch 6/20
55/55 [==============================] - 14s 249ms/step - loss: 3.6212 - accuracy: 0.2859 - val_loss: 11.9492 - val_accuracy: 0.4747
Epoch 7/20
55/55 [==============================] - 12s 218ms/step - loss: 3.1183 - accuracy: 0.2874 - val_loss: 9.8252 - val_accuracy

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42
)


model = BiLSTMCRFModel(
    vocab_size=len(word2idx),
    tag_size=len(label2idx),
    class_weights=class_weights_array
)
model.compile(optimizer='adam')

class NERF1Callback(tf.keras.callbacks.Callback):
    def __init__(self, model, X_val, y_val):
        self.model = model
        self.X_val = X_val
        self.y_val = y_val
        self.idx2label = {0: 'B-KNOW', 1: 'B-PRIN', 2: 'I-KNOW', 3: 'I-PRIN', 4: 'O'}

    def on_epoch_end(self, epoch, logs=None):
        logits = self.model(self.X_val, training=False)
        lengths = np.sum(self.X_val != 0, axis=1)
        viterbi_seq, _ = tfa.text.crf_decode(logits, self.model.transition_params, lengths)
        y_pred = viterbi_seq.numpy()
        y_true = self.y_val

        flat_y_true, flat_y_pred = [], []
        for i in range(len(y_true)):
            seq_len = lengths[i]
            flat_y_true += list(y_true[i][:seq_len])
            flat_y_pred += list(y_pred[i][:seq_len])

        flat_y_true_label = [self.idx2label[idx] for idx in flat_y_true]
        flat_y_pred_label = [self.idx2label[idx] for idx in flat_y_pred]

        labels_to_report = ['B-KNOW', 'B-PRIN', 'I-KNOW', 'I-PRIN']
        print("\n[VAL NER F1 SCORE]")
        print(classification_report(flat_y_true_label, flat_y_pred_label, labels=labels_to_report, digits=4))


# add F1 callback
f1_cb = NERF1Callback(model, X_val, y_val)
model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[f1_cb]
)


Epoch 1/20
55/55 [==============================] - ETA: 0s - loss: 26.2499 - accuracy: 0.2089
[VAL NER F1 SCORE]
              precision    recall  f1-score   support

      B-KNOW     0.6000    0.0054    0.0107      1111
      B-PRIN     0.0000    0.0000    0.0000        47
      I-KNOW     0.5907    0.6214    0.6057      2018
      I-PRIN     0.5517    0.1159    0.1916       138

   micro avg     0.5902    0.3850    0.4660      3314
   macro avg     0.4356    0.1857    0.2020      3314
weighted avg     0.5838    0.3850    0.3804      3314

55/55 [==============================] - 24s 294ms/step - loss: 26.4235 - accuracy: 0.2103 - val_loss: 22.2367 - val_accuracy: 0.2525
Epoch 2/20


c:\Users\Jiawei Qi\anaconda3\envs\umlt\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Jiawei Qi\anaconda3\envs\umlt\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Jiawei Qi\anaconda3\envs\umlt\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

55/55 [==============================] - ETA: 0s - loss: 12.0217 - accuracy: 0.2666
[VAL NER F1 SCORE]
              precision    recall  f1-score   support

      B-KNOW     0.7868    0.7309    0.7578      1111
      B-PRIN     1.0000    0.0213    0.0417        47
      I-KNOW     0.7789    0.8850    0.8286      2018
      I-PRIN     0.6667    0.5797    0.6202       138

   micro avg     0.7774    0.8084    0.7926      3314
   macro avg     0.8081    0.5542    0.5621      3314
weighted avg     0.7800    0.8084    0.7850      3314

55/55 [==============================] - 14s 264ms/step - loss: 11.9517 - accuracy: 0.2661 - val_loss: 9.8912 - val_accuracy: 0.2690
Epoch 3/20
55/55 [==============================] - ETA: 0s - loss: 7.3251 - accuracy: 0.2757
[VAL NER F1 SCORE]
              precision    recall  f1-score   support

      B-KNOW     0.8312    0.8110    0.8210      1111
      B-PRIN     0.8710    0.5745    0.6923        47
      I-KNOW     0.8392    0.9257    0.8803      2018

In [22]:
from collections import defaultdict

def extract_entities(labels):

    entities = []
    start, entity_type = None, None
    for i, tag in enumerate(labels):
        if tag.startswith("B-"):
            if start is not None:
                entities.append((start, i - 1, entity_type))
            start = i
            entity_type = tag[2:]
        elif tag.startswith("I-"):
            if start is not None and tag[2:] == entity_type:
                continue
            else:
                if start is not None:
                    entities.append((start, i - 1, entity_type))
                start = None
                entity_type = None
        else:  # tag == "O"
            if start is not None:
                entities.append((start, i - 1, entity_type))
                start = None
                entity_type = None
    if start is not None:
        entities.append((start, len(labels) - 1, entity_type))
    return set(entities)

def compute_f1(true_seqs, pred_seqs):

    assert len(true_seqs) == len(pred_seqs)
    total_true = 0
    total_pred = 0
    correct = 0

    for true_labels, pred_labels in zip(true_seqs, pred_seqs):
        true_entities = extract_entities(true_labels)
        pred_entities = extract_entities(pred_labels)

        total_true += len(true_entities)
        total_pred += len(pred_entities)
        correct += len(true_entities & pred_entities)

    precision = correct / total_pred if total_pred > 0 else 0
    recall = correct / total_true if total_true > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1


In [23]:
model.save_weights("bilstm_crf_weights.h5")


In [24]:

test_file_path = 'C:/Users/Jiawei Qi/MathsSelfLearnModel/dataset/SeniorMaths/test_data'

# Get test sentences
test_sentences = []
words = []
with open(test_file_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            if words:
                test_sentences.append(words)
                words = []
        else:
            try:
                word, _ = line.split()
                words.append(word)
            except ValueError:
                continue
# Capture the last sentence (if it doesn't end with an empty line)
if words:
    test_sentences.append(words)


In [25]:


test_X = [[word2idx.get(w, word2idx["UNK"]) for w in s] for s in test_sentences]
test_X = tf.keras.preprocessing.sequence.pad_sequences(
    test_X, maxlen=X.shape[1], padding="post", value=word2idx["PAD"]
)


In [26]:
# Forecast the labels using the model
logits = model.predict(test_X)

# CRF 
viterbi_seq, _ = tfa.text.crf_decode(
    logits,
    model.transition_params,
    tf.math.count_nonzero(test_X, axis=1)
)


9/9 [==============================] - 3s 38ms/step


In [27]:
# Create a mapping from label indices to label names
idx2label = {v: k for k, v in label2idx.items()}

# decoding
for i, sentence in enumerate(test_sentences):
    pred_tags = viterbi_seq.numpy()[i][:len(sentence)]
    tag_names = [idx2label.get(tag, "O") for tag in pred_tags]

    print("Sentense:", "".join(sentence))
    print("Tags:", tag_names)
    print("-" * 40)


Sentense: 一般的研究对象统称为元素，一些元素组成的总体叫集合，简称为集。
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O']
----------------------------------------
Sentense: 绝对补集定义：A关于全集合U的相对补集称作A的绝对补集，记作A'或∁u（A）或~A。有U'=Φ；Φ'=U。
Tags: ['B-KNOW', 'I-KNOW', 'I-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'I-KNOW', 'O', 'O', 'B-KNOW', 'I-KNOW', 'I-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'I-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
----------------------------------------
Sentense: 如果集合A的任何一个元素都是集合B的元素，我们说这两个集合有包含关系，称集合A是集合B的子集。
Tags: ['O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-KNOW', 'I-KNOW', 'O', 'O'

In [28]:
def extract_entities(words, labels):
    entities = {"KNOW": [], "PRIN": []}
    entity = []
    current_type = None

    for word, label in zip(words, labels):
        if label.startswith("B-"):
            if entity and current_type:
                entities[current_type].append("".join(entity))
            entity = [word]
            current_type = label[2:]
        elif label.startswith("I-") and current_type == label[2:]:
            entity.append(word)
        else:
            if entity and current_type:
                entities[current_type].append("".join(entity))
            entity = []
            current_type = None

    if entity and current_type:
        entities[current_type].append("".join(entity))

    return entities


In [29]:
# One Sentence
sentence = test_sentences[25]
pred_tags = viterbi_seq.numpy()[25][:len(sentence)]
tag_names = [idx2label.get(tag, "O") for tag in pred_tags]


In [30]:
entities = extract_entities(sentence, tag_names)
print("Knowledge (KNOW):", entities["KNOW"])
print("Principles (PRIN):", entities["PRIN"])


Knowledge (KNOW): ['向量', '加', '向量', '向量', '加法', '向量', '加', '向量', '减法向量']
Principles (PRIN): ['三角形法则', '平行四边形法则']


In [31]:
for i, sentence in enumerate(test_sentences):
    pred_tags = viterbi_seq.numpy()[i][:len(sentence)]
    tag_names = [idx2label.get(tag, "O") for tag in pred_tags]
    entities = extract_entities(sentence, tag_names)

    print(f" {i+1} ：{''.join(sentence)}")
    print("  📘 Knowledge:", entities["KNOW"])
    print("  📙 Principles:", entities["PRIN"])
    print()


 1 ：一般的研究对象统称为元素，一些元素组成的总体叫集合，简称为集。
  📘 Knowledge: ['元素', '元素组', '集合']
  📙 Principles: []

 2 ：绝对补集定义：A关于全集合U的相对补集称作A的绝对补集，记作A'或∁u（A）或~A。有U'=Φ；Φ'=U。
  📘 Knowledge: ['绝对补集', '全集合', '相对补集', '绝对补集']
  📙 Principles: []

 3 ：如果集合A的任何一个元素都是集合B的元素，我们说这两个集合有包含关系，称集合A是集合B的子集。
  📘 Knowledge: ['集合', '元素', '集合', '元素', '集合', '集合', '集合', '子集']
  📙 Principles: []

 4 ：任何一个集合是它本身的子集。
  📘 Knowledge: ['集合', '子集']
  📙 Principles: []

 5 ：空集是任何集合的子集，空集是任何非空集合的真子集。
  📘 Knowledge: ['空集', '集合', '子集', '空集', '非空集合', '真子集']
  📙 Principles: []

 6 ：集合A中的每一个元素，在集合B中都有象，并且象是唯一的，集合A中不同的元素，在集合B中对应的象可以是同一个。
  📘 Knowledge: ['集合', '元素', '集合', '集合', '元素', '集合']
  📙 Principles: []

 7 ：如果函数y=f(x)在某个区间是增函数或减函数，那么说函数y=f(x)在这一区间上具有(严格的)单调性。
  📘 Knowledge: ['函数', '区间', '增函数', '减函数', '函数', '区间', '单调性']
  📙 Principles: []

 8 ：偶函数的图象关于y轴对称；奇函数的图象关于原点对称。
  📘 Knowledge: ['偶函数', 'y轴', '奇函数', '原点', '对称']
  📙 Principles: []

 9 ：在公共定义域内，偶函数的加减乘除仍为偶函数；奇函数的加减仍为奇函数；奇数个奇函数的乘除认为奇函数；偶数个奇函数的乘除为偶函数；一奇一偶的乘积是奇函数。
  📘 Knowledge: ['定义域', '偶函数'

In [32]:
# Set max_len
max_len = 100  # ⚠️ 

def predict_custom_sentence(sentence):
    tokens = list(sentence.strip())
    UNK_ID = word2idx.get("UNK", 1)
    input_ids = [word2idx.get(w, UNK_ID) for w in tokens]

    input_ids_padded = pad_sequences([input_ids], maxlen=max_len, padding='post', value=0)

    logits = model.predict(input_ids_padded)

    # Use CRF decoding 
    from tensorflow_addons.text import crf_decode
    sequence_lengths = np.array([len(tokens)])
    viterbi_seq, _ = crf_decode(logits, model.transition_params, sequence_lengths)

    pred_tags = viterbi_seq.numpy()[0][:len(tokens)]
    tag_names = [idx2label.get(tag, "O") for tag in pred_tags]

    print("Sentence:", "".join(tokens))
    print("Tags:", tag_names)

    entities = extract_entities(tokens, tag_names)
    print("Knowledge Point (KNOW):", entities["KNOW"])
    print("Principle (PRIN):", entities["PRIN"])
    print("-" * 40)

# Test
input_sentence = input("Please input one sentense：")
predict_custom_sentence(input_sentence)


1/1 [==============================] - 2s 2s/step
Sentence: 三 角 形 的 数 形 结 合 法
Tags: ['B-KNOW', 'I-KNOW', 'I-KNOW', 'O', 'B-PRIN', 'I-PRIN', 'I-PRIN', 'I-PRIN', 'I-PRIN']
Knowledge Point (KNOW): ['三角形']
Principle (PRIN): ['数形结合法']
----------------------------------------
